Today you are an IoT consulatant and you need to make a decision beased on the results you see in this notebook about the effect of the number of clients and the dataset size. 


The research question here is :
Would you advise a company with a low number of clients and small dataset to adopt FL?
How would you argue your decision? can you visualise? 

For the below experiments write down the experiment setup ( the parameter values ,dataset, etc) 

Extra Questions: 

Would more rounds help? does it always help? 

are clients who go for fitting always chosen for evaluation?
are the same clients chosen every round?


In [1]:
!pip install -U --pre flwr["simulation"] tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import math
# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import flwr as fl
import tensorflow as tf

In [3]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
    
    def get_parameters(self, config):
        return self.model.get_weights()
    #step 2 + 3
    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=3, verbose=2)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        #print("lala"+ str(self.x_val.size()))
        self.model.set_weights(parameters)
        
        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        return  loss, len(self.x_val), {"accuracy": acc}

In [4]:

def client_fn(cid: str) -> fl.client.Client:
    # Create model
    """
    ONLY use this for the mnist
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )
    """
    model = tf.keras.applications.VGG16( input_shape=(32, 32, 3), weights=None, classes=10  )
    model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    
    (x_train, y_train), _ = tf.keras.datasets.cifar10.load_data()
    partition_size = math.floor(len(x_train) / NUM_CLIENTS)#100
    idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size#200,300
    x_train_cid = x_train[idx_from:idx_to] / 255.0 #normalize
    y_train_cid = y_train[idx_from:idx_to]
   
    
    split_idx = math.floor(len(x_train_cid) * 0.9)
    
    xx= x_train_cid
    yy= y_train_cid
    x_train_cid, y_train_cid = xx[:split_idx], yy[:split_idx]
    x_val_cid, y_val_cid = xx[split_idx:], yy[split_idx:]
   
    
    return FlowerClient(model, x_train_cid, y_train_cid, x_val_cid, y_val_cid)

## ***50 Clients***

In [ ]:
NUM_CLIENTS = 50

# Create FedAvg strategy
strategy=fl.server.strategy.FedAvg(
        fraction_fit=0.1,  # Sample 10% of available clients for training
        fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation     
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)


INFO flower 2022-10-26 08:49:09,716 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO flower 2022-10-26 08:49:13,474 | app.py:179 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'CPU': 4.0, 'memory': 16056673076.0, 'accelerator_type:T4': 1.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 8028336537.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'CPU': 4.0, 'memory': 16056673076.0, 'accelerator_type:T4': 1.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 8028336537.0}
INFO flower 2022-10-26 08:49:13,481 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-10-26 08:49:13,483 | server.py:270 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client


(launch_and_get_parameters pid=298) Downloading data from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
170498071/170498071 [==============================] - 11s 0us/step


INFO flower 2022-10-26 08:49:30,666 | server.py:274 | Received initial parameters from one random client
INFO:flower:Received initial parameters from one random client
INFO flower 2022-10-26 08:49:30,672 | server.py:88 | Evaluating initial parameters
INFO:flower:Evaluating initial parameters
INFO flower 2022-10-26 08:49:30,677 | server.py:101 | FL starting
INFO:flower:FL starting
DEBUG flower 2022-10-26 08:49:30,683 | server.py:220 | fit_round 1: strategy sampled 5 clients (out of 50)
DEBUG:flower:fit_round 1: strategy sampled 5 clients (out of 50)


(launch_and_fit pid=298) Epoch 1/3
(launch_and_fit pid=297) Epoch 1/3
(launch_and_fit pid=295) Epoch 1/3
(launch_and_fit pid=296) Epoch 1/3
(launch_and_fit pid=298) 29/29 - 160s - loss: 2.3049 - accuracy: 0.1111 - 160s/epoch - 6s/step
(launch_and_fit pid=298) Epoch 2/3
(launch_and_fit pid=296) 29/29 - 165s - loss: 2.3047 - accuracy: 0.0800 - 165s/epoch - 6s/step
(launch_and_fit pid=296) Epoch 2/3
(launch_and_fit pid=295) 29/29 - 166s - loss: 2.3052 - accuracy: 0.1033 - 166s/epoch - 6s/step
(launch_and_fit pid=295) Epoch 2/3
(launch_and_fit pid=297) 29/29 - 168s - loss: 2.3596 - accuracy: 0.0856 - 168s/epoch - 6s/step
(launch_and_fit pid=297) Epoch 2/3
(launch_and_fit pid=298) 29/29 - 161s - loss: 2.2999 - accuracy: 0.1033 - 161s/epoch - 6s/step
(launch_and_fit pid=298) Epoch 3/3
(launch_and_fit pid=296) 29/29 - 159s - loss: 2.3027 - accuracy: 0.0844 - 159s/epoch - 5s/step
(launch_and_fit pid=296) Epoch 3/3
(launch_and_fit pid=295) 29/29 - 158s - loss: 2.3027 - accuracy: 0.1178 - 158s/e

DEBUG flower 2022-10-26 08:59:43,762 | server.py:234 | fit_round 1 received 5 results and 0 failures
DEBUG:flower:fit_round 1 received 5 results and 0 failures
WARNING flower 2022-10-26 08:59:44,859 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-10-26 08:59:44,882 | server.py:170 | evaluate_round 1: strategy sampled 2 clients (out of 50)
DEBUG:flower:evaluate_round 1: strategy sampled 2 clients (out of 50)
DEBUG flower 2022-10-26 08:59:47,738 | server.py:184 | evaluate_round 1 received 2 results and 0 failures
DEBUG:flower:evaluate_round 1 received 2 results and 0 failures
WARNING flower 2022-10-26 08:59:47,741 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-10-26 08:59:47,745 | server.py:220 | fit_round 2: strategy sampled 5 clients (out of 50)
DEBUG:flower:fit_round 2: strategy sampled 5 clients (out of 50)


...

(launch_and_evaluate pid=298) 4/4 - 1s - loss: 2.3036 - accuracy: 0.1000 - 1s/epoch - 323ms/step
(launch_and_evaluate pid=295) 4/4 - 1s - loss: 2.3079 - accuracy: 0.1100 - 1s/epoch - 322ms/step
(launch_and_fit pid=295) Epoch 1/3
(launch_and_fit pid=297) Epoch 1/3
(launch_and_fit pid=298) Epoch 1/3
(launch_and_fit pid=296) Epoch 1/3
(launch_and_fit pid=297) 29/29 - 156s - loss: 2.3005 - accuracy: 0.1078 - 156s/epoch - 5s/step
(launch_and_fit pid=297) Epoch 2/3
(launch_and_fit pid=298) 29/29 - 161s - loss: 2.3016 - accuracy: 0.0956 - 161s/epoch - 6s/step
(launch_and_fit pid=298) Epoch 2/3
(launch_and_fit pid=296) 29/29 - 162s - loss: 2.3043 - accuracy: 0.0889 - 162s/epoch - 6s/step
(launch_and_fit pid=296) Epoch 2/3
(launch_and_fit pid=295) 29/29 - 163s - loss: 2.3032 - accuracy: 0.1011 - 163s/epoch - 6s/step
(launch_and_fit pid=295) Epoch 2/3
(launch_and_fit pid=297) 29/29 - 156s - loss: 2.2969 - accuracy: 0.1244 - 156s/epoch - 5s/step
(launch_and_fit pid=297) Epoch 3/3
(launch_and_fit 

DEBUG flower 2022-10-26 09:40:20,614 | server.py:234 | fit_round 5 received 5 results and 0 failures
DEBUG:flower:fit_round 5 received 5 results and 0 failures
DEBUG flower 2022-10-26 09:40:21,384 | server.py:170 | evaluate_round 5: strategy sampled 2 clients (out of 50)
DEBUG:flower:evaluate_round 5: strategy sampled 2 clients (out of 50)


(launch_and_evaluate pid=296) 4/4 - 1s - loss: 2.3087 - accuracy: 0.0900 - 1s/epoch - 326ms/step


DEBUG flower 2022-10-26 09:40:24,885 | server.py:184 | evaluate_round 5 received 2 results and 0 failures
DEBUG:flower:evaluate_round 5 received 2 results and 0 failures
INFO flower 2022-10-26 09:40:24,891 | server.py:144 | FL finished in 3054.20851688
INFO:flower:FL finished in 3054.20851688
INFO flower 2022-10-26 09:40:24,906 | app.py:180 | app_fit: losses_distributed [(1, 2.3020354509353638), (2, 2.3007701635360718), (3, 2.3015851974487305), (4, 2.305739164352417), (5, 2.3063130378723145)]
INFO:flower:app_fit: losses_distributed [(1, 2.3020354509353638), (2, 2.3007701635360718), (3, 2.3015851974487305), (4, 2.305739164352417), (5, 2.3063130378723145)]
INFO flower 2022-10-26 09:40:24,909 | app.py:181 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-10-26 09:40:24,915 | app.py:182 | app_fit: losses_centralized []
INFO:flower:app_fit: losses_centralized []
INFO flower 2022-10-26 09:40:24,917 | app.py:183 | app_fit: metrics_centralized {}
IN

History (loss, distributed):
	round 1: 2.3020354509353638
	round 2: 2.3007701635360718
	round 3: 2.3015851974487305
	round 4: 2.305739164352417
	round 5: 2.3063130378723145

## ***100 Clients***

In [5]:
NUM_CLIENTS = 100

# Create FedAvg strategy
strategy=fl.server.strategy.FedAvg(
        fraction_fit=0.1,  # Sample 10% of available clients for training
        fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation     
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)


INFO flower 2022-10-27 03:40:12,549 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO flower 2022-10-27 03:40:15,201 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 16084937934.0, 'GPU': 1.0, 'CPU': 4.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 8042468966.0, 'accelerator_type:T4': 1.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'memory': 16084937934.0, 'GPU': 1.0, 'CPU': 4.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 8042468966.0, 'accelerator_type:T4': 1.0}
INFO flower 2022-10-27 03:40:15,209 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-10-27 03:40:15,213 | server.py:270 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client


(launch_and_get_parameters pid=309) Downloading data from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
170498071/170498071 [==============================] - 11s 0us/step


INFO flower 2022-10-27 03:40:32,757 | server.py:274 | Received initial parameters from one random client
INFO:flower:Received initial parameters from one random client
INFO flower 2022-10-27 03:40:32,760 | server.py:88 | Evaluating initial parameters
INFO:flower:Evaluating initial parameters
INFO flower 2022-10-27 03:40:32,763 | server.py:101 | FL starting
INFO:flower:FL starting
DEBUG flower 2022-10-27 03:40:32,766 | server.py:220 | fit_round 1: strategy sampled 10 clients (out of 100)
DEBUG:flower:fit_round 1: strategy sampled 10 clients (out of 100)


(launch_and_fit pid=309) Epoch 1/3
(launch_and_fit pid=306) Epoch 1/3
(launch_and_fit pid=308) Epoch 1/3
(launch_and_fit pid=307) Epoch 1/3
(launch_and_fit pid=309) 15/15 - 82s - loss: 6.3916 - accuracy: 0.1044 - 82s/epoch - 5s/step
(launch_and_fit pid=309) Epoch 2/3
(launch_and_fit pid=306) 15/15 - 88s - loss: 3.5465 - accuracy: 0.1156 - 88s/epoch - 6s/step
(launch_and_fit pid=306) Epoch 2/3
(launch_and_fit pid=307) 15/15 - 87s - loss: 2.4708 - accuracy: 0.1222 - 87s/epoch - 6s/step
(launch_and_fit pid=307) Epoch 2/3
(launch_and_fit pid=308) 15/15 - 89s - loss: 2.3057 - accuracy: 0.1067 - 89s/epoch - 6s/step
(launch_and_fit pid=308) Epoch 2/3
(launch_and_fit pid=309) 15/15 - 82s - loss: 2.3002 - accuracy: 0.1333 - 82s/epoch - 5s/step
(launch_and_fit pid=309) Epoch 3/3
(launch_and_fit pid=307) 15/15 - 81s - loss: 2.3056 - accuracy: 0.0978 - 81s/epoch - 5s/step
(launch_and_fit pid=307) Epoch 3/3
(launch_and_fit pid=306) 15/15 - 83s - loss: 2.2966 - accuracy: 0.1089 - 83s/epoch - 6s/step

DEBUG flower 2022-10-27 03:51:10,733 | server.py:234 | fit_round 1 received 10 results and 0 failures
DEBUG:flower:fit_round 1 received 10 results and 0 failures
WARNING flower 2022-10-27 03:51:13,312 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-10-27 03:51:13,356 | server.py:170 | evaluate_round 1: strategy sampled 5 clients (out of 100)
DEBUG:flower:evaluate_round 1: strategy sampled 5 clients (out of 100)


...

DEBUG flower 2022-10-27 04:34:41,866 | server.py:234 | fit_round 5 received 10 results and 0 failures
DEBUG:flower:fit_round 5 received 10 results and 0 failures
DEBUG flower 2022-10-27 04:34:44,343 | server.py:170 | evaluate_round 5: strategy sampled 5 clients (out of 100)
DEBUG:flower:evaluate_round 5: strategy sampled 5 clients (out of 100)
(launch_and_evaluate pid=308) WARNING:tensorflow:5 out of the last 9 calls to <function Model.make_test_function.<locals>.test_function at 0x7f28aefd0dd0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https:

(launch_and_evaluate pid=308) 2/2 - 1s - loss: 2.3021 - accuracy: 0.1200 - 1s/epoch - 749ms/step
(launch_and_evaluate pid=309) 2/2 - 1s - loss: 2.3005 - accuracy: 0.1200 - 1s/epoch - 742ms/step
(launch_and_evaluate pid=306) 2/2 - 2s - loss: 2.3026 - accuracy: 0.1400 - 2s/epoch - 774ms/step
(launch_and_evaluate pid=307) 2/2 - 2s - loss: 2.3099 - accuracy: 0.0800 - 2s/epoch - 756ms/step


(launch_and_evaluate pid=308) WARNING:tensorflow:6 out of the last 11 calls to <function Model.make_test_function.<locals>.test_function at 0x7f28b11e2d40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
(launch_and_evaluate pid=308) WARNING:tensorflow:6 out of the last 11 calls to <function Model.make_test_function.<locals>.test_function at 0x7f28b11e2d40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @

(launch_and_evaluate pid=308) 2/2 - 1s - loss: 2.3028 - accuracy: 0.0600 - 583ms/epoch - 291ms/step


History (loss, distributed):
	round 1: 2.302608871459961
	round 2: 2.303168296813965
	round 3: 2.302614450454712
	round 4: 2.3029120922088624
	round 5: 2.303599214553833

## ***200 Clients***

In [5]:
NUM_CLIENTS = 200

# Create FedAvg strategy
strategy=fl.server.strategy.FedAvg(
        fraction_fit=0.1,  # Sample 10% of available clients for training
        fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation     
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)


INFO flower 2022-10-27 05:46:11,809 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO flower 2022-10-27 05:46:13,619 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 4.0, 'GPU': 1.0, 'memory': 16262342247.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 8131171123.0, 'accelerator_type:T4': 1.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'CPU': 4.0, 'GPU': 1.0, 'memory': 16262342247.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 8131171123.0, 'accelerator_type:T4': 1.0}
INFO flower 2022-10-27 05:46:13,625 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-10-27 05:46:13,628 | server.py:270 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
INFO flower 2022-10-27 05:46:18,000 | serve

(launch_and_fit pid=9617) Epoch 1/3
(launch_and_fit pid=9616) Epoch 1/3
(launch_and_fit pid=9618) Epoch 1/3
(launch_and_fit pid=9615) Epoch 1/3
(launch_and_fit pid=9617) 8/8 - 42s - loss: 2.3083 - accuracy: 0.1022 - 42s/epoch - 5s/step
(launch_and_fit pid=9617) Epoch 2/3
(launch_and_fit pid=9618) 8/8 - 48s - loss: 2.3075 - accuracy: 0.0978 - 48s/epoch - 6s/step
(launch_and_fit pid=9618) Epoch 2/3
(launch_and_fit pid=9615) 8/8 - 48s - loss: 2.3031 - accuracy: 0.1111 - 48s/epoch - 6s/step
(launch_and_fit pid=9615) Epoch 2/3
(launch_and_fit pid=9616) 8/8 - 49s - loss: 2.3100 - accuracy: 0.1333 - 49s/epoch - 6s/step
(launch_and_fit pid=9616) Epoch 2/3
(launch_and_fit pid=9617) 8/8 - 44s - loss: 2.3013 - accuracy: 0.1333 - 44s/epoch - 5s/step
(launch_and_fit pid=9617) Epoch 3/3
(launch_and_fit pid=9618) 8/8 - 42s - loss: 2.3020 - accuracy: 0.0978 - 42s/epoch - 5s/step
(launch_and_fit pid=9618) Epoch 3/3
(launch_and_fit pid=9615) 8/8 - 43s - loss: 2.2934 - accuracy: 0.1422 - 43s/epoch - 5s/s

DEBUG flower 2022-10-27 05:58:12,761 | server.py:234 | fit_round 1 received 20 results and 0 failures
DEBUG:flower:fit_round 1 received 20 results and 0 failures
WARNING flower 2022-10-27 05:58:17,614 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-10-27 05:58:17,741 | server.py:170 | evaluate_round 1: strategy sampled 10 clients (out of 200)
DEBUG:flower:evaluate_round 1: strategy sampled 10 clients (out of 200)


(launch_and_evaluate pid=9615) 1/1 - 1s - loss: 2.3021 - accuracy: 0.2000 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9617) 1/1 - 1s - loss: 2.3026 - accuracy: 0.0000e+00 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9618) 1/1 - 1s - loss: 2.3024 - accuracy: 0.2000 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9616) 1/1 - 1s - loss: 2.3025 - accuracy: 0.1600 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9615) 1/1 - 1s - loss: 2.3022 - accuracy: 0.2800 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9616) 1/1 - 1s - loss: 2.3025 - accuracy: 0.0800 - 970ms/epoch - 970ms/step
(launch_and_evaluate pid=9617) 1/1 - 1s - loss: 2.3028 - accuracy: 0.1200 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9618) 1/1 - 1s - loss: 2.3022 - accuracy: 0.0800 - 1s/epoch - 1s/step


DEBUG flower 2022-10-27 05:58:27,439 | server.py:184 | evaluate_round 1 received 10 results and 0 failures
DEBUG:flower:evaluate_round 1 received 10 results and 0 failures
WARNING flower 2022-10-27 05:58:27,444 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-10-27 05:58:27,447 | server.py:220 | fit_round 2: strategy sampled 20 clients (out of 200)
DEBUG:flower:fit_round 2: strategy sampled 20 clients (out of 200)


...

DEBUG flower 2022-10-27 06:46:35,031 | server.py:234 | fit_round 5 received 20 results and 0 failures
DEBUG:flower:fit_round 5 received 20 results and 0 failures
DEBUG flower 2022-10-27 06:46:40,233 | server.py:170 | evaluate_round 5: strategy sampled 10 clients (out of 200)
DEBUG:flower:evaluate_round 5: strategy sampled 10 clients (out of 200)


(launch_and_evaluate pid=9618) 1/1 - 1s - loss: 2.3038 - accuracy: 0.0800 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9616) 1/1 - 1s - loss: 2.3018 - accuracy: 0.0800 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9615) 1/1 - 1s - loss: 2.3020 - accuracy: 0.0400 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9617) 1/1 - 1s - loss: 2.3018 - accuracy: 0.1600 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9618) 1/1 - 1s - loss: 2.3022 - accuracy: 0.0400 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9617) 1/1 - 1s - loss: 2.3030 - accuracy: 0.1200 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9615) 1/1 - 1s - loss: 2.3022 - accuracy: 0.1600 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9616) 1/1 - 1s - loss: 2.3026 - accuracy: 0.0800 - 1s/epoch - 1s/step
(launch_and_evaluate pid=9618) 1/1 - 1s - loss: 2.3016 - accuracy: 0.1200 - 534ms/epoch - 534ms/step


DEBUG flower 2022-10-27 06:46:50,428 | server.py:184 | evaluate_round 5 received 10 results and 0 failures
DEBUG:flower:evaluate_round 5 received 10 results and 0 failures
INFO flower 2022-10-27 06:46:50,431 | server.py:144 | FL finished in 3632.421158274001
INFO:flower:FL finished in 3632.421158274001
INFO flower 2022-10-27 06:46:50,604 | app.py:180 | app_fit: losses_distributed [(1, 2.302344799041748), (2, 2.302490210533142), (3, 2.30235435962677), (4, 2.302501893043518), (5, 2.302431416511536)]
INFO:flower:app_fit: losses_distributed [(1, 2.302344799041748), (2, 2.302490210533142), (3, 2.30235435962677), (4, 2.302501893043518), (5, 2.302431416511536)]
INFO flower 2022-10-27 06:46:50,612 | app.py:181 | app_fit: metrics_distributed {}
INFO:flower:app_fit: metrics_distributed {}
INFO flower 2022-10-27 06:46:50,617 | app.py:182 | app_fit: losses_centralized []
INFO:flower:app_fit: losses_centralized []
INFO flower 2022-10-27 06:46:50,624 | app.py:183 | app_fit: metrics_centralized {}
IN

(launch_and_evaluate pid=9615) 1/1 - 1s - loss: 2.3034 - accuracy: 0.1200 - 538ms/epoch - 538ms/step


History (loss, distributed):
	round 1: 2.302344799041748
	round 2: 2.302490210533142
	round 3: 2.30235435962677
	round 4: 2.302501893043518
	round 5: 2.302431416511536

# ***Section 2***
### `cifar100`

In [8]:

def client_fn2(cid: str) -> fl.client.Client:
    # Create model
    """
    ONLY use this for the mnist
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )
    """
    model = tf.keras.applications.VGG16( input_shape=(32, 32, 3), weights=None, classes=100  )
    model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    
    (x_train, y_train), _ = tf.keras.datasets.cifar100.load_data()
    partition_size = math.floor(len(x_train) / NUM_CLIENTS)#100
    idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size#200,300
    x_train_cid = x_train[idx_from:idx_to] / 255.0 #normalize
    y_train_cid = y_train[idx_from:idx_to]
   
    
    split_idx = math.floor(len(x_train_cid) * 0.9)
    
    xx= x_train_cid
    yy= y_train_cid
    x_train_cid, y_train_cid = xx[:split_idx], yy[:split_idx]
    x_val_cid, y_val_cid = xx[split_idx:], yy[split_idx:]
   
    
    return FlowerClient(model, x_train_cid, y_train_cid, x_val_cid, y_val_cid)

In [9]:
NUM_CLIENTS = 50

# Create FedAvg strategy
strategy=fl.server.strategy.FedAvg(
        fraction_fit=0.1,  # Sample 10% of available clients for training
        fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation     
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn2,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)

INFO flower 2022-10-27 06:51:21,737 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2022-10-27 06:51:25,546	WARNING services.py:2013 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 4820877312 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=8.33gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
INFO flower 2022-10-27 06:51:28,421 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:172.28.0.2': 1.0, 'CPU': 4.0, 'GPU': 1.0, 'memory': 16268820480.0, 'accelerator_type:T4': 1.0, 'object_store_memory': 8134410240.0}
INFO:flower:Flower VCE: Ray initialized with 

(launch_and_get_parameters pid=13202) Downloading data from https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
169001437/169001437 [==============================] - 11s 0us/step


INFO flower 2022-10-27 06:51:51,707 | server.py:274 | Received initial parameters from one random client
INFO:flower:Received initial parameters from one random client
INFO flower 2022-10-27 06:51:51,709 | server.py:88 | Evaluating initial parameters
INFO:flower:Evaluating initial parameters
INFO flower 2022-10-27 06:51:51,713 | server.py:101 | FL starting
INFO:flower:FL starting
DEBUG flower 2022-10-27 06:51:51,719 | server.py:220 | fit_round 1: strategy sampled 5 clients (out of 50)
DEBUG:flower:fit_round 1: strategy sampled 5 clients (out of 50)


(launch_and_fit pid=13202) Epoch 1/3
(launch_and_fit pid=13204) Epoch 1/3
(launch_and_fit pid=13203) Epoch 1/3
(launch_and_fit pid=13205) Epoch 1/3
(launch_and_fit pid=13202) 29/29 - 164s - loss: 4.6088 - accuracy: 0.0111 - 164s/epoch - 6s/step
(launch_and_fit pid=13202) Epoch 2/3
(launch_and_fit pid=13204) 29/29 - 170s - loss: 4.6116 - accuracy: 0.0089 - 170s/epoch - 6s/step
(launch_and_fit pid=13204) Epoch 2/3
(launch_and_fit pid=13203) 29/29 - 171s - loss: 4.6105 - accuracy: 0.0133 - 171s/epoch - 6s/step
(launch_and_fit pid=13203) Epoch 2/3
(launch_and_fit pid=13205) 29/29 - 170s - loss: 4.6081 - accuracy: 0.0078 - 170s/epoch - 6s/step
(launch_and_fit pid=13205) Epoch 2/3
(launch_and_fit pid=13202) 29/29 - 167s - loss: 4.5881 - accuracy: 0.0200 - 167s/epoch - 6s/step
(launch_and_fit pid=13202) Epoch 3/3
(launch_and_fit pid=13205) 29/29 - 164s - loss: 4.5874 - accuracy: 0.0133 - 164s/epoch - 6s/step
(launch_and_fit pid=13205) Epoch 3/3
(launch_and_fit pid=13203) 29/29 - 164s - loss: 

DEBUG flower 2022-10-27 07:02:22,045 | server.py:234 | fit_round 1 received 5 results and 0 failures
DEBUG:flower:fit_round 1 received 5 results and 0 failures
WARNING flower 2022-10-27 07:02:23,128 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-10-27 07:02:23,146 | server.py:170 | evaluate_round 1: strategy sampled 2 clients (out of 50)
DEBUG:flower:evaluate_round 1: strategy sampled 2 clients (out of 50)
DEBUG flower 2022-10-27 07:02:26,259 | server.py:184 | evaluate_round 1 received 2 results and 0 failures
DEBUG:flower:evaluate_round 1 received 2 results and 0 failures
WARNING flower 2022-10-27 07:02:26,265 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-10-27 07:02:26,269 | server.py:220 | fit_round 2: strategy sampled 5 clients (out of 50)
DEBUG:flower:fit_round 2: strategy sampled 5 clients (out of 50)


...

DEBUG flower 2022-10-27 07:33:41,330 | server.py:234 | fit_round 4 received 5 results and 0 failures
DEBUG:flower:fit_round 4 received 5 results and 0 failures
DEBUG flower 2022-10-27 07:33:42,199 | server.py:170 | evaluate_round 4: strategy sampled 2 clients (out of 50)
DEBUG:flower:evaluate_round 4: strategy sampled 2 clients (out of 50)
DEBUG flower 2022-10-27 07:33:45,346 | server.py:184 | evaluate_round 4 received 2 results and 0 failures
DEBUG:flower:evaluate_round 4 received 2 results and 0 failures
DEBUG flower 2022-10-27 07:33:45,351 | server.py:220 | fit_round 5: strategy sampled 5 clients (out of 50)
DEBUG:flower:fit_round 5: strategy sampled 5 clients (out of 50)


(launch_and_evaluate pid=13203) 4/4 - 1s - loss: 4.6132 - accuracy: 0.0100 - 1s/epoch - 327ms/step
(launch_and_evaluate pid=13205) 4/4 - 1s - loss: 4.6270 - accuracy: 0.0000e+00 - 1s/epoch - 345ms/step
(launch_and_fit pid=13205) Epoch 1/3
(launch_and_fit pid=13204) Epoch 1/3
(launch_and_fit pid=13203) Epoch 1/3
(launch_and_fit pid=13202) Epoch 1/3
(launch_and_fit pid=13203) 29/29 - 171s - loss: 4.6115 - accuracy: 0.0089 - 171s/epoch - 6s/step
(launch_and_fit pid=13203) Epoch 2/3
(launch_and_fit pid=13204) 29/29 - 171s - loss: 4.6096 - accuracy: 0.0111 - 171s/epoch - 6s/step
(launch_and_fit pid=13204) Epoch 2/3
(launch_and_fit pid=13205) 29/29 - 171s - loss: 4.6119 - accuracy: 0.0133 - 171s/epoch - 6s/step
(launch_and_fit pid=13205) Epoch 2/3
(launch_and_fit pid=13202) 29/29 - 171s - loss: 4.6077 - accuracy: 0.0089 - 171s/epoch - 6s/step
(launch_and_fit pid=13202) Epoch 2/3
(launch_and_fit pid=13203) 29/29 - 166s - loss: 4.5974 - accuracy: 0.0167 - 166s/epoch - 6s/step
(launch_and_fit p

DEBUG flower 2022-10-27 07:44:16,892 | server.py:234 | fit_round 5 received 5 results and 0 failures
DEBUG:flower:fit_round 5 received 5 results and 0 failures
DEBUG flower 2022-10-27 07:44:17,763 | server.py:170 | evaluate_round 5: strategy sampled 2 clients (out of 50)
DEBUG:flower:evaluate_round 5: strategy sampled 2 clients (out of 50)
DEBUG flower 2022-10-27 07:44:20,859 | server.py:184 | evaluate_round 5 received 2 results and 0 failures
DEBUG:flower:evaluate_round 5 received 2 results and 0 failures
INFO flower 2022-10-27 07:44:20,862 | server.py:144 | FL finished in 3149.1433862699996
INFO:flower:FL finished in 3149.1433862699996
INFO flower 2022-10-27 07:44:20,885 | app.py:180 | app_fit: losses_distributed [(1, 4.605322360992432), (2, 4.609693765640259), (3, 4.609066724777222), (4, 4.620133876800537), (5, 4.61467432975769)]
INFO:flower:app_fit: losses_distributed [(1, 4.605322360992432), (2, 4.609693765640259), (3, 4.609066724777222), (4, 4.620133876800537), (5, 4.614674329757

(launch_and_evaluate pid=13202) 4/4 - 1s - loss: 4.6118 - accuracy: 0.0000e+00 - 1s/epoch - 336ms/step
(launch_and_evaluate pid=13205) 4/4 - 1s - loss: 4.6175 - accuracy: 0.0100 - 1s/epoch - 328ms/step


History (loss, distributed):
	round 1: 4.605322360992432
	round 2: 4.609693765640259
	round 3: 4.609066724777222
	round 4: 4.620133876800537
	round 5: 4.61467432975769

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
    
    def get_parameters(self, config):
        return self.model.get_weights()
    #step 2 + 3
    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=10, verbose=2)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        #print("lala"+ str(self.x_val.size()))
        self.model.set_weights(parameters)
        
        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        return  loss, len(self.x_val), {"accuracy": acc}

NUM_CLIENTS = 50

# Create FedAvg strategy
strategy=fl.server.strategy.FedAvg(
        fraction_fit=0.1,  # Sample 10% of available clients for training
        fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation     
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn2,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)

INFO flower 2022-10-27 07:46:11,103 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO flower 2022-10-27 07:46:17,165 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 8133428428.0, 'CPU': 4.0, 'GPU': 1.0, 'memory': 16266856859.0, 'node:172.28.0.2': 1.0, 'accelerator_type:T4': 1.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'object_store_memory': 8133428428.0, 'CPU': 4.0, 'GPU': 1.0, 'memory': 16266856859.0, 'node:172.28.0.2': 1.0, 'accelerator_type:T4': 1.0}
INFO flower 2022-10-27 07:46:17,170 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-10-27 07:46:17,175 | server.py:270 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
INFO flower 2022-10-27 07:46:21,545 | serve

(launch_and_fit pid=15234) Epoch 1/10
(launch_and_fit pid=15233) Epoch 1/10
(launch_and_fit pid=15235) Epoch 1/10
(launch_and_fit pid=15236) Epoch 1/10
